In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import logging
logging.basicConfig(level=logging.INFO, format='')
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import seaborn as sns
import rdkit
from sklearn.impute import SimpleImputer
from scipy.stats import kurtosis, skew
from rdkit import Chem, RDLogger
from rdkit.Chem import Draw
from rdkit.Chem import BondType
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem import PandasTools
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.ML.Descriptors import MoleculeDescriptors
import requests
import torch
import pubchempy as pcp
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import SVG, display
from fast_molvae.sample import load_model
model = load_model('../data/vocab.txt', '../fast_molvae/vae_model/model.epoch-19')
torch.cuda.is_available()
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
import random
import joblib
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['font.sans-serif'] = ["Arial"]
import warnings
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Enabling RDKit 2020.09.1 jupyter extensions
C:\ProgramData\Anaconda3\envs\chemvae2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\ProgramData\Anaconda3\envs\chemvae2\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [2]:
#Plt True vs Pred
def prediction_vs_ground_truth_fig(y_train, y_train_hat, y_test, y_test_hat):
    from sklearn import metrics
    fontsize = 16
    lims = (13, 25)
    plt.figure(figsize=(6,5.5))
    plt.style.use('default')
    plt.rc('xtick', labelsize=fontsize)
    plt.rc('ytick', labelsize=fontsize)
    plt.rcParams['font.family']="Arial"
    a = plt.scatter(y_train, y_train_hat, s=20,lw=1.5,c='lightseagreen')
    plt.plot(lims, lims, 'k--', alpha=0.75, linewidth =1.5,zorder=0)
    plt.xlabel('True PCE[%]', fontsize=fontsize)
    plt.ylabel('Predicted PCE[%]', fontsize=fontsize)
    plt.tick_params(direction='in',length=4, width=1, labelsize = fontsize*.85)
    plt.xlim([13,25]) 
    plt.ylim([13,25])
    plt.title(('Train RMSE: {:.2}'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_hat))),\
           'Test RMSE: {:.2}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_test_hat)))), fontsize=fontsize)
    b = plt.scatter(y_test, y_test_hat, s=20,lw=1.5,c='sandybrown')
    plt.legend((a,b),('Train','Test'),fontsize=fontsize,handletextpad=0.1,borderpad=0.1)
    plt.rcParams['font.family']="Arial"

    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(2.5)
    
    plt.tight_layout()
    plt.show()
    
    print ('RMSE Train: %.2f' % (np.sqrt(mse(y_train, y_train_hat))))
    print ('RMSE Test: %.2f' % (np.sqrt(mse(y_test, y_test_hat))))
    print('^'*20)
    print ('MAE Train: %.2f' % (mean_absolute_error(y_train, y_train_hat)))
    print ('MAE Test: %.2f' % (mean_absolute_error(y_test, y_test_hat)))
    print('^'*20)
    print ('R2 Train: %.2f' % (r2_score(y_train, y_train_hat)))
    print ('R2 Test: %.2f' % (r2_score(y_test, y_test_hat)))
    print('^'*20)
    print ('Spearman Train: %.2f' % (spearmanr(y_train, y_train_hat)[0]))
    print ('Spearman Test: %.2f' % (spearmanr(y_test, y_test_hat)[0]))
    print('^'*20)

In [3]:
pd.set_option('display.max_rows',None)
data=pd.read_csv('./molecule_data.csv',encoding='gb18030')
data.head()

,Condition #,smiles,Control_PCE,PCE_difference,target_PCE
0,0,Cn1nnnc1S,19.20,1.30,20.50
1,1,I.Nc1ccccc1,19.20,1.30,20.50
2,2,NCC=Cc1ccccc1,17.37,4.61,21.98
3,3,CSCC[C@H](NC(=O)OCC1c2ccccc2c3ccccc13)C(O)=O,14.17,2.58,16.75
4,4,c1ccncc1,13.34,1.00,14.34


In [4]:
#Generate Scaffold
smi_list=data.iloc[:,1].tolist()
mol_list=[]
for i in range(len(smi_list)):
    smile = smi_list[i]
    mol = Chem.MolFromSmiles(smile)
    mol_list.append(mol)
scaffold_list=[]
for i in range(len(mol_list)):
    scaffold=Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(mol_list[i]))
    scaffold_list.append(scaffold)
indices = {}
for index, item in enumerate(scaffold_list):
    if item in indices:
        indices[item].append(index)
    else:
        indices[item] = [index]
print(indices)

{'c1nnn[nH]1': [0], 'c1ccccc1': [1, 2, 7, 8, 9, 14, 15, 16, 22, 24, 32, 33, 39, 40, 42, 59, 62, 63, 65, 67, 68, 75, 80, 82, 84], 'c1ccc2c(c1)Cc1ccccc1-2': [3], 'c1ccncc1': [4, 5, 6, 19, 21, 71, 72], 'c1ccc2c(c1)c1ccccc1n2-c1ccncc1': [10], 'c1ccc(-c2ccnc3c2ccc2c(-c4ccccc4)ccnc23)cc1': [11], 'c1cscn1': [12], '': [13, 17, 20, 23, 28, 41, 44, 46, 48, 51, 52, 54, 55, 56, 57, 58, 60, 61, 76, 77, 78, 79], 'O=S(Cc1ccccn1)c1nc2ccccc2[nH]1': [18], 'O=c1[nH]c(=O)c2[nH]cnc2[nH]1': [25, 26, 38], 'O=P(c1ccccc1)(c1ccccc1)c1ccccc1': [27], 'c1ccc(P(c2ccccc2)c2ccccc2)cc1': [29], 'C1CCCCC1': [30], 'c1ccc2ccccc2c1': [31, 64, 73], 'c1cnc2c(c1)ccc1cccnc12': [34], 'c1ccc(-c2cccs2)cc1': [35, 36, 37], 'O=C1C=CC(=O)N1': [43], 'O=c1ccc2cc3c4c(c2o1)CCCN4CCC3': [45], 'O=C1c2ccc3c4cccc5cccc(c6ccc(c2c36)C(=O)N1c1ccccc1)c54': [47], 'O=c1[nH]sc2ccccc12': [49], 'O=c1cc[nH]c(=O)[nH]1': [50], 'C=c1ccc(=C)cc1': [53], 'c1ncc2ncn([C@H]3CCCO3)c2n1': [66], 'O=C(NCCSCc1ccccc1)OCC1c2ccccc2-c2ccccc21': [69], 'c1csc[nH+]1': [70],

In [5]:
#Input as D+M
molecule_smi= data.iloc[:,1]
descriptors = ['FractionCSP3', 'BCUT2D_MRLOW', 'fr_quatN', 'PEOE_VSA8', 'fr_Ar_N',
       'MinEStateIndex', 'fr_NH0', 'fr_imidazole', 'fr_bicyclic',
       'fr_para_hydroxylation', 'BCUT2D_MWHI', 'PEOE_VSA2',
       'NumSaturatedHeterocycles', 'BalabanJ', 'SlogP_VSA11',
       'NumAliphaticHeterocycles', 'PEOE_VSA5', 'MaxAbsEStateIndex',
       'NumAromaticHeterocycles', 'MaxAbsPartialCharge', 'VSA_EState3',
       'VSA_EState6', 'VSA_EState2', 'HallKierAlpha', 'PEOE_VSA9',
       'fr_aniline', 'fr_pyridine', 'VSA_EState4', 'PEOE_VSA1', 'Chi4v',
       'fr_C_O', 'EState_VSA4', 'SlogP_VSA3', 'MolLogP', 'fr_Nhpyrrole',
       'Chi1v', 'fr_NH1', 'EState_VSA3', 'PEOE_VSA7', 'fr_ether', 'SMR_VSA10',
       'SlogP_VSA8', 'SMR_VSA1', 'HeavyAtomMolWt']

molecule_mols = [Chem.MolFromSmiles(smi) for smi in molecule_smi]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors)
result = [desc_calc.CalcDescriptors(mol) for mol in molecule_mols]
desc_df = pd.DataFrame(result)
desc_df.columns = descriptors
imputer = SimpleImputer(missing_values=np.nan, strategy='mean', verbose=0)
imputer.fit(desc_df)
desc = imputer.transform(desc_df)

df_maccs =  pd.read_csv('./MACCS_data.csv',encoding='gb18030')
maccs=df_maccs.drop('Unnamed: 0', axis=1)

dm = np.hstack((desc,maccs))

In [6]:
# Group Scaffold
scaffold_groups = [
    ['c1nnn[nH]1', 'c1ccccc1','c1ccncc1','c1cscn1','c1ccsc1','C=c1ccc(=C)cc1','c1ccsc1','c1csc[nH+]1',
    'c1ncncn1','C1=NCCN1','c1c[nH]cn1','c1cc[nH+]cc1','S=c1cccc[nH]1'],
    ['c1ccc2c(c1)Cc1ccccc1-2', 'c1ccc2c(c1)c1ccccc1n2-c1ccncc1', 'c1ccc(-c2ccnc3c2ccc2c(-c4ccccc4)ccnc23)cc1',
     'O=S(Cc1ccccn1)c1nc2ccccc2[nH]1','c1ccc2ccccc2c1','c1cnc2c(c1)ccc1cccnc12','O=c1ccc2cc3c4c(c2o1)CCCN4CCC3',
    'O=C1c2ccc3c4cccc5cccc(c6ccc(c2c36)C(=O)N1c1ccccc1)c54','c1ncc2ncn([C@H]3CCCO3)c2n1','O=C(NCCSCc1ccccc1)OCC1c2ccccc2-c2ccccc21',
    'O=c1ccc2ccccc2o1','c1ccc2c(c1)[nH]c1ccccc12'],
    ['','C1CCCCC1'],
    ['O=c1[nH]c(=O)c2[nH]cnc2[nH]1', 'O=C1C=CC(=O)N1','O=c1[nH]sc2ccccc12','O=c1cc[nH]c(=O)[nH]1','O=c1nc[nH]c2nc[nH]c12','O=c1nccc[nH]1',
     'O=c1ccc2ccccc2[nH]1','O=C1/C(=C2/Nc3ccccc3C2=O)Nc2ccccc21','O=C(Nc1ccccc1)Nc1ccccc1','O=C1C=CC(=O)N1c1ccccc1','C1=CC2(Cc3ccccc3N2)Oc2ccccc21',
     'O=C1C=CC(=O)N1c1ccc(Cc2ccc(N3C(=O)C=CC3=O)cc2)cc1','O=C(c1ccccc1)c1ccccc1','O=C(/C=C/C=C/c1ccc2c(c1)OCO2)N1CCCCC1','O=c1cccc[nH]1'],
    ['c1ccc(-c2cccs2)cc1', 'c1ccc(-c2ccccn2)nc1','c1cnc2ncccc2c1','c1csc(-c2ccc(-c3cccs3)c3nsnc23)c1','c1ccc2scnc2c1','c1ccc(Cc2ccccc2)cc1',
     'S=c1[nH]c2ccccc2[nH]1','c1ccc(-n2nc3ccccc3n2)cc1','c1ccc2[nH]ccc2c1','O=S1(=O)N=Cc2ccccc21','c1csc(-c2cccs2)c1','c1ccc(-c2ccccc2)cc1',
     'c1ccc2[nH]cnc2c1','O=C1NC(=O)C(c2ccsc2)=C1c1ccsc1'],
    ['O=P(c1ccccc1)(c1ccccc1)c1ccccc1', 'c1ccc(P(c2ccccc2)c2ccccc2)cc1'],
    ['C1COCCN1','O=C1COCCN1','C1C[NH2+]CCN1','O=C1CC(=O)NC(=O)N1','O=C1CCC(=O)N1','C1CCOCC1','c1ccoc1'],
    ['C1C2CC3CC1CC(C2)C3','O=C1C2CC3CC(C2)CC1C3'],
    ['c1ccc(-c2nc(-c3ccccc3)nc(-c3ccccc3)n2)cc1']
]
    
# Create a dictionary to map each molecule to its scaffold group
scaffold_mapping = {}
for idx, scaffolds in enumerate(scaffold_groups):
    for scaffold in scaffolds:
        scaffold_mapping[scaffold] = idx

# Map each molecule to its corresponding scaffold
def map_scaffold(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        scaffold = Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(mol))
        return scaffold_mapping.get(scaffold, -1)
    return -1

data['scaffold_group'] = data['smiles'].apply(map_scaffold)
for idx, group in enumerate(scaffold_groups):
    count = data[data['scaffold_group'] == idx].shape[0]
    print(f"Scaffold Group {idx}: {count} molecules")

Scaffold Group 0: 43 molecules
Scaffold Group 1: 14 molecules
Scaffold Group 2: 23 molecules
Scaffold Group 3: 17 molecules
Scaffold Group 4: 18 molecules
Scaffold Group 5: 2 molecules
Scaffold Group 6: 7 molecules
Scaffold Group 7: 3 molecules
Scaffold Group 8: 2 molecules


In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import joblib

# Initialize empty DataFrame to store metrics
metrics_list = []

# 9 scaffold group indices
scaffold_group_indices = list(range(len(scaffold_groups)))

# Loop over each scaffold group (9 iterations)
for test_group_idx in tqdm(scaffold_group_indices, desc="Folds"):
    # Initialize train and test sets
    train_data = data[data['scaffold_group'] != test_group_idx]
    test_data = data[data['scaffold_group'] == test_group_idx]

    # Scale data
    X_scaler = MinMaxScaler()
    X_train = X_scaler.fit_transform(dm[train_data.index])
    X_test = X_scaler.transform(dm[test_data.index])

    y_train = np.transpose([train_data['target_PCE'].values])
    y_test = np.transpose([test_data['target_PCE'].values])
    
    print(f"Current test set size ({test_group_idx}):", len(y_test))

    # Train GradientBoostingRegressor
    gb_regressor = GradientBoostingRegressor(n_estimators=35, max_depth=4)
    gb_regressor.fit(X_train, y_train)

    # Predict on the test set
    y_test_pred = gb_regressor.predict(X_test)

    # Calculate RMSE, MAE, Spearman, R2
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    mae = mean_absolute_error(y_test, y_test_pred)
    spearman_corr, _ = spearmanr(y_test.flatten(), y_test_pred.flatten())
    r2 = r2_score(y_test, y_test_pred)
    
    # Append metrics to the list
    metrics_list.append({
        'RMSE': rmse,
        'MAE': mae,
        'Spearman': spearman_corr,
        'R2': r2
    })

# Convert list of metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate mean and std of metrics across all 9 folds
mean_rmse = np.mean(metrics_df['RMSE'])
std_rmse = np.std(metrics_df['RMSE'])
mean_mae = np.mean(metrics_df['MAE'])
std_mae = np.std(metrics_df['MAE'])
mean_spearman = np.mean(metrics_df['Spearman'])
std_spearman = np.std(metrics_df['Spearman'])
mean_r2 = np.mean(metrics_df['R2'])
std_r2 = np.std(metrics_df['R2'])

# Save summary of metrics
summary_df = pd.DataFrame({
    'Metric': ['RMSE', 'MAE', 'Spearman', 'R2'],
    'Mean': [mean_rmse, mean_mae, mean_spearman, mean_r2],
    'Std': [std_rmse, std_mae, std_spearman, std_r2]
})

# Save results to CSV files
metrics_df.to_csv('metrics_GB_9DM.csv', index=False)
summary_df.to_csv('metrics_summary_GB_9DMs.csv', index=False)

# Print summary
print(summary_df)

当前测试集数量 (0): 43
当前测试集数量 (1): 14
当前测试集数量 (2): 23
当前测试集数量 (3): 17
当前测试集数量 (4): 18
当前测试集数量 (5): 2
当前测试集数量 (6): 7
当前测试集数量 (7): 3
当前测试集数量 (8): 2
     Metric      Mean        Std
0      RMSE  2.206883   0.668300
1       MAE  1.812600   0.487434
2  Spearman  0.024110   0.607047
3        R2 -8.333534  18.498172
